In [1]:
#Import basic packages
import os
import numpy as np
import pandas as pd
import csv
import regex as re
import datetime as dt 

In [2]:
#import outcome file with org_uuid
main_df = pd.read_csv(R"d:/msc-project/data/final/outcome_final_v1.csv",encoding='utf-8',
                         index_col=False,
                         parse_dates= ['founded_on','seed_date','series_a_date','series_b_date','series_c_date'])
main_processing_df = main_df[['org_uuid']]

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
main_processing_df.head(3)

,org_uuid
0,eb37f7c8-c036-a915-bb7c-63f445330f66
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60
2,de462c42-b0a5-c5af-9637-ec426b4e991f


In [4]:
len(main_processing_df)

38605

In [5]:
#import jobs file
jobs_df = pd.read_csv(R"d:/msc-project/data/pre-processed/jobs_preprocessed.csv",encoding='utf-8',
                         index_col=False,
                         parse_dates= ['started_on','ended_on'])
jobs_df.head(3)

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder
0,697b6934-fc1f-9d63-cfb2-1a10759b378e,Ben Elowitz Co-Founder/CEO/Board of Directors ...,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben Elowitz,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,NaN,False,Co-Founder/CEO/Board of Directors,executive,1
1,b1de3765-442e-b556-9304-551c2a055901,Kevin Flaherty VP Marketing @ Wetpaint,5ceca97b-493c-1446-6249-5aaa33464763,Kevin Flaherty,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,NaN,False,VP Marketing,executive,0
2,1319cd30-f5e8-c700-0af6-64029c6f7124,Raju Vegesna Chief Evangelist @ Zoho,9f99a98a-aa97-b30b-0d36-db67c1d277e0,Raju Vegesna,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,2000-11-01,NaN,True,Chief Evangelist,employee,0


# Assign Number of founder for each organisation

In [6]:
num_founder = pd.DataFrame(jobs_df.groupby('org_uuid')['is_founder'].sum())
main_processing_df_v1 = pd.merge(main_processing_df,
                         num_founder,
                         on='org_uuid',
                         how='left'
                         )
main_processing_df_v1.rename(columns = {'is_founder':'num_founder'},inplace=True)
main_processing_df_v1.head(3)

,org_uuid,num_founder
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0.0
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,1.0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,1.0


In [7]:
main_processing_df_v1[main_processing_df_v1['num_founder'].isna()]

,org_uuid,num_founder
6,88206b5f-eefe-e06c-5a84-0dedd98dd529,NaN
12,722fd5dc-f201-fe36-551e-09895bba7d6e,NaN
19,e4d0dca6-64e0-a455-fb34-4d86e9099daa,NaN
22,2a2ddece-2eb4-96b7-8cce-03cd7facb61d,NaN
31,0a4f84dd-3f55-0846-ee4e-ed8eceeb43ef,NaN
...,...,...
38596,993a321e-5895-42c2-a234-841891fd694b,NaN
38598,b9779f2f-f9d9-5729-570c-f2db17247c20,NaN
38599,207d0f85-34b0-45d4-8bf5-63841ae0ec4b,NaN
38600,e5c41b77-ca63-46f8-a1c5-ab0ffa60696d,NaN


In [8]:
#drop companies with no founder info
main_processing_df_v1.dropna(subset=['num_founder'],inplace=True)
main_processing_df_v2 = main_processing_df_v1[main_processing_df_v1['num_founder'] > 0]

In [9]:
len(main_processing_df_v2)

26259

In [10]:
main_processing_df_v2.head(5)

,org_uuid,num_founder
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,1.0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,1.0
5,1b923b7c-b415-a83c-cbcc-53b27c877907,1.0
7,91f5ca18-e026-9482-8e4f-22593943e0f5,3.0
8,3d607a21-39bb-651e-c53f-9890ee334f18,1.0


# Assign Gender Diversity Features

In [11]:
founder_df = jobs_df[jobs_df['is_founder'] == 1]
founder_gb = founder_df.groupby('org_uuid')

In [12]:
people_df = pd.read_csv(R"d:/msc-project/data/pre-processed/people_preprocessed.csv",encoding='utf-8',
                         index_col=False)

In [13]:
people_df.columns

Index(['people_uuid', 'people_name', 'first_name', 'last_name', 'gender',
       'country_code', 'state_code', 'region', 'city', 'org_uuid',
       'featured_job_organization_name', 'featured_job_title', 'facebook_url',
       'linkedin_url', 'twitter_url', 'logo_url', 'description'],
      dtype='object')

In [14]:
founder_gender_df = pd.merge(founder_df[['people_uuid','org_uuid']],
                             people_df[['people_uuid','gender']],
                             on='people_uuid',
                             how='left')

In [15]:
founder_gender_df.head(3)

,people_uuid,org_uuid,gender
0,ed13cd36-fe2b-3707-197b-0c2d56e37a71,e1393508-30ea-8a36-3f96-dd3226033abd,male
1,a01b8d46-d311-3333-7c34-aa3ae9c03f22,df662812-7f97-0b43-9d3e-12f64f504fbb,male
2,084aaa07-0795-1fe8-9c46-98bbeb02cd64,df662812-7f97-0b43-9d3e-12f64f504fbb,male


In [16]:
founder_gender_gb = founder_gender_df.groupby('org_uuid')

In [17]:
is_founder_gender_diversity = list()

for org_uuid in list(main_processing_df_v2['org_uuid']):
    founders_gender = list(founder_gender_gb.get_group(org_uuid)['gender']) #create list of founder's gender for one organisation
    if 'male' in founders_gender and 'female' in founders_gender: #assign 1 if male and female in gender list
        is_founder_gender_diversity.append(1)
    else:
        is_founder_gender_diversity.append(0)

main_processing_df_v2['is_founder_gender_diversity'] = is_founder_gender_diversity

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [18]:
len(main_processing_df_v2)

26259

In [19]:
main_processing_df_v2.head(3)

,org_uuid,num_founder,is_founder_gender_diversity
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,1.0,0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,1.0,0
5,1b923b7c-b415-a83c-cbcc-53b27c877907,1.0,0


# Assigning degree type, ivy league, and top100 to organisation based on founder

In [20]:
#import processed degree file
deg_df = pd.read_csv(R"d:/msc-project/data/pre-processed/degrees_preprocessed.csv",encoding='utf-8',
                         index_col=False)
deg_df.head(3)

,deg_uuid,deg_name,people_uuid,person_name,institution_uuid,institution_name,degree_type,subject,started_on,completed_on,is_completed,is_bachelor,is_master,is_phd,is_mba,is_stem,is_ivy_league,is_top_100
0,205fdfd1-ecac-aa43-262f-219f11755f67,MS Mass Communication @ Boston University,4897dba9-3141-ecc0-2c4b-c9d844e6440f,John Green,1eab62d2-15d9-0db7-930f-2aa77d4688e1,Boston University,MS,Mass Communication,NaN,1992-01-01,True,0,1,0,0,1,0,1
1,1a2ac288-eb99-3318-fde5-1517bc168f51,"BA English, French @ Washington University in...",4897dba9-3141-ecc0-2c4b-c9d844e6440f,John Green,6ae9957a-8fb4-0ab1-73fa-dd547c4d3da4,Washington University in St. Louis,BA,"English, French",NaN,1990-01-01,True,1,0,0,0,0,0,0
2,b978d338-7ccc-7469-5ce7-ef98c34155ad,MS Internet Technology @ University of Greenwich,7d187b77-94f7-e6cc-6981-d7468db5968f,Sridhar Gundaiah,b5ea73f6-12a3-576d-ae9b-f4169147f974,University of Greenwich,MS,Internet Technology,NaN,2006-01-01,True,0,1,0,0,1,0,0


In [197]:
#merge degree with founder_df
deg_type_df = deg_df[['people_uuid','is_bachelor','is_master','is_phd','is_mba','is_stem','is_ivy_league','is_top_100']]
founder_degree_df = pd.merge(founder_df[['people_uuid','org_uuid']],
                             deg_type_df,
                             on='people_uuid',
                             how='left')
founder_degree_df.fillna(value=0,inplace=True)
len(founder_degree_df)

355966

In [198]:
#group by org_uuid and merge with main file
founder_degree_df = founder_degree_df[['org_uuid','is_bachelor','is_master','is_phd','is_mba','is_stem',
                                      'is_ivy_league','is_top_100']].groupby('org_uuid').sum()

founder_degree_df.reset_index(inplace=True)

main_processing_df_v3 = pd.merge(main_processing_df_v2,
                                 founder_degree_df,
                                 on='org_uuid',
                                 how = 'left')

In [199]:
main_processing_df_v3['is_bachelor'].value_counts()

0.0     19777
1.0      4763
2.0      1320
3.0       303
4.0        72
5.0        18
6.0         3
7.0         2
11.0        1
Name: is_bachelor, dtype: int64

In [200]:
main_processing_df_v3['is_phd'].value_counts()

0.0    24836
1.0     1230
2.0      153
3.0       27
4.0       11
5.0        2
Name: is_phd, dtype: int64

In [24]:
#assign 0 to founder with no degree information
main_processing_df_v3.isnull().sum(axis=0)

org_uuid                       0
num_founder                    0
is_founder_gender_diversity    0
is_bachelor                    0
is_master                      0
is_phd                         0
is_mba                         0
is_stem                        0
is_ivy_league                  0
is_top_100                     0
dtype: int64

In [25]:
main_processing_df_v3.head(10)

,org_uuid,num_founder,is_founder_gender_diversity,is_bachelor,is_master,is_phd,is_mba,is_stem,is_ivy_league,is_top_100
0,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,de462c42-b0a5-c5af-9637-ec426b4e991f,1.0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1b923b7c-b415-a83c-cbcc-53b27c877907,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,91f5ca18-e026-9482-8e4f-22593943e0f5,3.0,0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
4,3d607a21-39bb-651e-c53f-9890ee334f18,1.0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,2a5fa1db-4075-04fb-d76c-372158a3a2c8,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0f19649c-4fb9-cc37-d9b9-753d738c9da7,1.0,0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
7,7635f325-b7c2-8d0d-b428-df100ae21e8b,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,229afe91-4a19-4f02-86a5-88a948c350a6,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,ec74826e-c72e-a898-d676-f710724dd73d,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Assign number of advisor

In [26]:
jobs_df['job_type'].value_counts()

executive         967719
employee          395731
board_member      154586
advisor            57657
board_observer      5948
Name: job_type, dtype: int64

In [27]:
jobs_advisor_df = jobs_df[jobs_df['job_type'] == 'advisor']

In [28]:
jobs_advisor_df.head(3)

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder
247,b8902477-3f1d-4147-5bca-f053b3491b75,Brad Gerstner Investor / Advisor / Board Membe...,46daadf8-9ab9-1f94-24e2-f83ad6d4ad7f,Brad Gerstner,5c6f4d9e-dc72-e018-ee21-71cd23f698ad,Farecast,2006-01-01,2008-01-01,False,Investor / Advisor / Board Member,advisor,0
353,8986d170-77a9-a918-fe22-b8de983d1e42,Scott Rafer Advisor @ Dogster,eaadb11b-7423-0d5d-0a9d-7bf93eb64579,Scott Rafer,b683af1f-8dcf-8f82-37d9-88fbadd63e1b,Dogster,NaN,NaN,True,Advisor,advisor,0
444,70e011ab-472a-9f50-77fe-718af9ae9fe6,George Martin Advisor @ iLike,651ba47c-9aef-f811-a39d-d917677aded4,George Martin,3091d3e8-7934-1e90-7c82-7f1c3caf53cb,iLike,NaN,NaN,False,Advisor,advisor,0


In [29]:
advisor_gb = jobs_advisor_df.groupby('org_uuid')

In [30]:
num_advisor = advisor_gb.size()
num_advisor = pd.DataFrame(num_advisor,
                          columns=['num_advisor'])

In [31]:
main_processing_df_v4 = pd.merge(main_processing_df_v3,
                                 num_advisor,
                                 on='org_uuid',
                                 how='left')

In [32]:
main_processing_df_v4.head(3)

,org_uuid,num_founder,is_founder_gender_diversity,is_bachelor,is_master,is_phd,is_mba,is_stem,is_ivy_league,is_top_100,num_advisor
0,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,de462c42-b0a5-c5af-9637-ec426b4e991f,1.0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
2,1b923b7c-b415-a83c-cbcc-53b27c877907,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [33]:
main_processing_df_v4['num_advisor'].fillna(0,inplace=True)

In [34]:
main_processing_df_v4.head(3)

,org_uuid,num_founder,is_founder_gender_diversity,is_bachelor,is_master,is_phd,is_mba,is_stem,is_ivy_league,is_top_100,num_advisor
0,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,de462c42-b0a5-c5af-9637-ec426b4e991f,1.0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1b923b7c-b415-a83c-cbcc-53b27c877907,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Work Experience

In [40]:
main_processing_df_v4.head(3)

,org_uuid,num_founder,is_founder_gender_diversity,is_bachelor,is_master,is_phd,is_mba,is_stem,is_ivy_league,is_top_100,num_advisor
0,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,de462c42-b0a5-c5af-9637-ec426b4e991f,1.0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1b923b7c-b415-a83c-cbcc-53b27c877907,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
jobs_df.head(3)

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder
0,697b6934-fc1f-9d63-cfb2-1a10759b378e,Ben Elowitz Co-Founder/CEO/Board of Directors ...,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben Elowitz,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,NaN,False,Co-Founder/CEO/Board of Directors,executive,1
1,b1de3765-442e-b556-9304-551c2a055901,Kevin Flaherty VP Marketing @ Wetpaint,5ceca97b-493c-1446-6249-5aaa33464763,Kevin Flaherty,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,NaN,False,VP Marketing,executive,0
2,1319cd30-f5e8-c700-0af6-64029c6f7124,Raju Vegesna Chief Evangelist @ Zoho,9f99a98a-aa97-b30b-0d36-db67c1d277e0,Raju Vegesna,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,2000-11-01,NaN,True,Chief Evangelist,employee,0


In [42]:
jobs_df_v1= jobs_df [['job_uuid','people_uuid','org_uuid','started_on','ended_on','is_current']]

In [43]:
jobs_df_v1.head(3)

,job_uuid,people_uuid,org_uuid,started_on,ended_on,is_current
0,697b6934-fc1f-9d63-cfb2-1a10759b378e,ed13cd36-fe2b-3707-197b-0c2d56e37a71,e1393508-30ea-8a36-3f96-dd3226033abd,NaN,NaN,False
1,b1de3765-442e-b556-9304-551c2a055901,5ceca97b-493c-1446-6249-5aaa33464763,e1393508-30ea-8a36-3f96-dd3226033abd,NaN,NaN,False
2,1319cd30-f5e8-c700-0af6-64029c6f7124,9f99a98a-aa97-b30b-0d36-db67c1d277e0,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,2000-11-01,NaN,True


In [44]:
jobs_df_v2 = jobs_df_v1.dropna(subset=['started_on'])
jobs_df_v2.isnull().sum(axis=0)

job_uuid            0
people_uuid         0
org_uuid            0
started_on          0
ended_on       505002
is_current          0
dtype: int64

In [45]:
jobs_df_v3 = jobs_df_v2[jobs_df_v2['started_on'] > '1980-01-01']

In [46]:
jobs_df_v3['started_on'] = pd.to_datetime(jobs_df_v3['started_on'])
jobs_df_v3['ended_on'] = pd.to_datetime(jobs_df_v3['ended_on'])

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
founder_df.head(3)

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder
0,697b6934-fc1f-9d63-cfb2-1a10759b378e,Ben Elowitz Co-Founder/CEO/Board of Directors ...,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben Elowitz,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,NaN,False,Co-Founder/CEO/Board of Directors,executive,1
7,befc22de-c789-2096-e4d6-919935cf4204,Mark Zuckerberg Founder & CEO @ Facebook,a01b8d46-d311-3333-7c34-aa3ae9c03f22,Mark Zuckerberg,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,2004-01-01,NaN,True,Founder & CEO,executive,1
8,9ef8b964-186e-55e9-5faa-fb1d60034f6e,Dustin Moskovitz Co-Founder @ Facebook,084aaa07-0795-1fe8-9c46-98bbeb02cd64,Dustin Moskovitz,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,2004-02-01,2008-11-01,False,Co-Founder,executive,1


In [48]:
founder_df_v1 = pd.merge(founder_df,
                         main_df[['org_uuid','founded_on']],
                         on='org_uuid',
                         how='left')

In [49]:
founder_df_v1.head(3)

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder,founded_on
0,697b6934-fc1f-9d63-cfb2-1a10759b378e,Ben Elowitz Co-Founder/CEO/Board of Directors ...,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben Elowitz,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,NaN,False,Co-Founder/CEO/Board of Directors,executive,1,2005-06-01
1,befc22de-c789-2096-e4d6-919935cf4204,Mark Zuckerberg Founder & CEO @ Facebook,a01b8d46-d311-3333-7c34-aa3ae9c03f22,Mark Zuckerberg,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,2004-01-01,NaN,True,Founder & CEO,executive,1,2004-02-04
2,9ef8b964-186e-55e9-5faa-fb1d60034f6e,Dustin Moskovitz Co-Founder @ Facebook,084aaa07-0795-1fe8-9c46-98bbeb02cd64,Dustin Moskovitz,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,2004-02-01,2008-11-01,False,Co-Founder,executive,1,2004-02-04


In [50]:
len(founder_df_v1)

332569

In [81]:
founder_df_v2 = pd.merge(founder_df_v1,
                        main_processing_df_v4['org_uuid'],
                        on = 'org_uuid',
                        how='right')

In [52]:
len(founder_df_v2)

48562

In [53]:
founder_df_v2.head(3)

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder,founded_on
0,c6dbe07f-1dcf-6730-c9a7-17ebbf41d7e9,Cedric Paillard Director of Product Line and F...,4438fa50-fdd1-e19e-8f3c-231d835af288,Cedric Paillard,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,IceFyre Semiconductor,NaN,NaN,False,Director of Product Line and Founder,executive,1,2001-01-01
1,e5531b3a-84f5-74af-aa2b-7c769a36559f,Wim Keppens Co-founder @ ORMvision,bf9479dc-d542-6a47-9590-ac1ca9bd43d8,Wim Keppens,de462c42-b0a5-c5af-9637-ec426b4e991f,ORMvision,2001-07-01,2005-12-01,False,Co-founder,executive,1,2000-12-01
2,6ec5857a-7d73-fe76-df10-a0041f275ed2,"Christopher Anzalone Founder, CEO @ NanoInk",700d0f89-3a0e-e625-d807-ee4f8bc14a03,Christopher Anzalone,1b923b7c-b415-a83c-cbcc-53b27c877907,NanoInk,NaN,NaN,False,"Founder, CEO",executive,1,2001-01-01


In [54]:
founder_gb = founder_df_v2.groupby('org_uuid')

In [ ]:
founder_experience_num_companies = list()
founder_experience_is_successfull = list ()

In [179]:
%%time
#get features from founders' experience
founder_experience_years = list()

#successful_companies = main_df[main_df['outcome'] == 1]
#successful_companies_uuid = list(successful_companies['org_uuid'])

jobs_gb = jobs_df_v3.groupby('people_uuid')

end = len(founder_df_v2)

for i in range (0,end):
    founder = founder_df_v2.iloc[i]
    founded_date = founder['founded_on']
    people_uuid = founder['people_uuid']
    org_uuid = founder['org_uuid']
    
    if people_uuid not in list(jobs_df_v3['people_uuid']): #check if people_uuid is not in jobs_df (founder has no recorded experience)
        founder_experience_years.append(0)
        #founder_experience_num_companies.append(0)
        #founder_experience_is_successfull.append(0)
    
    else:
        founder_experience_df =  jobs_gb.get_group(people_uuid) #create pd of jobs for one founder
        #founder_experience_df = founder_experience_df[founder_experience_df['org_uuid'] != org_uuid] #drop the company founded
        founder_experience_df = founder_experience_df[founder_experience_df['started_on'] < founded_date] #filter to experience only before found company

        if len(founder_experience_df) == 0: #no experience found after filtering
            founder_experience_years.append(0)
            #founder_experience_num_companies.append(0)
            #founder_experience_is_successfull.append(0)
        else:
            #compute years of experience
            total_experience_years = round ((founded_date - founder_experience_df['started_on'].min()).days/365,1) 
            founder_experience_years.append(total_experience_years) #append years of experience to main list

            #compute number of companies
            #num_companies = len(founder_experience_df) #calculate number of companies worked before founded_on
            #founder_experience_num_companies.append(num_companies) #append to main list

            #if org_uuid in successful_companies_uuid:
                #founder_experience_is_successfull.append(1)
            #else:
                #founder_experience_is_successfull.append(0)


KeyboardInterrupt: 

In [150]:
jobs_gb['people_uuid']

In [139]:
total_experience_years

1.4

In [152]:
founder_experience_years.count(0)

504

In [180]:
len(founder_experience_years)

32719

In [177]:
founder_df_v2.iloc[9]

job_uuid                      8dacd62f-6103-3cb0-5b81-f294991ab633
job_name                    Michael Sherman Co-Founder @ VaxInnate
people_uuid                   c4bf6ffc-049c-709f-3e8f-8a790fe24e7e
person_name                                        Michael Sherman
org_uuid                      7635f325-b7c2-8d0d-b428-df100ae21e8b
org_name                                                 VaxInnate
started_on                                                     NaN
ended_on                                                       NaN
is_current                                                   False
title                                                   Co-Founder
job_type                                                 executive
is_founder                                                       1
founded_on                                     2002-01-01 00:00:00
experience_years                                               0.0
experience_num_companies                                      

In [171]:
founded_date = founder_df_v2.iloc[8]['founded_on']
founded_date

Timestamp('2002-01-01 00:00:00')

In [178]:
founder_experience = jobs_gb.get_group('c4bf6ffc-049c-709f-3e8f-8a790fe24e7e')
founder_experience

,job_uuid,people_uuid,org_uuid,started_on,ended_on,is_current
437054,efea0a81-2e09-6e7c-a77e-f48fe938252c,c4bf6ffc-049c-709f-3e8f-8a790fe24e7e,4e4677ed-9924-286e-c6ea-c2001e6b9d01,2002-01-01,2007-04-01,False


In [175]:
founder_experience['started_on'].min()

Timestamp('2001-01-01 00:00:00')

In [176]:
(founded_date - founder_experience['started_on'].min()).days/365

1.0

In [83]:
founder_df_v2['experience_years'] = founder_experience_years
founder_df_v2['experience_num_companies'] = founder_experience_num_companies
founder_df_v2['is_previously_successful'] = founder_experience_is_successfull

In [78]:
founder_df_v2.head(3)

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder,founded_on,experience_years,experience_num_companies,is_previously_successful
0,c6dbe07f-1dcf-6730-c9a7-17ebbf41d7e9,Cedric Paillard Director of Product Line and F...,4438fa50-fdd1-e19e-8f3c-231d835af288,Cedric Paillard,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,IceFyre Semiconductor,NaN,NaN,False,Director of Product Line and Founder,executive,1,2001-01-01 00:00:00,0.0,0,0
1,e5531b3a-84f5-74af-aa2b-7c769a36559f,Wim Keppens Co-founder @ ORMvision,bf9479dc-d542-6a47-9590-ac1ca9bd43d8,Wim Keppens,de462c42-b0a5-c5af-9637-ec426b4e991f,ORMvision,2001-07-01,2005-12-01,False,Co-founder,executive,1,2000-12-01 00:00:00,0.0,0,0
2,6ec5857a-7d73-fe76-df10-a0041f275ed2,"Christopher Anzalone Founder, CEO @ NanoInk",700d0f89-3a0e-e625-d807-ee4f8bc14a03,Christopher Anzalone,1b923b7c-b415-a83c-cbcc-53b27c877907,NanoInk,NaN,NaN,False,"Founder, CEO",executive,1,2001-01-01 00:00:00,0.0,0,0


In [84]:
founder_df_v2['experience_years'].max()

159.5

In [85]:
founder_df_v2[founder_df_v2['experience_years'] < 0]

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder,founded_on,experience_years,experience_num_companies,is_previously_successful
26629,9ef203bf-09ad-2596-ef69-65b31da2e8fc,Aron Jones Co-Founder & CTO @ Shopcaster,0b8daa42-f550-e8c9-fbdc-c366ae8b8f38,Aron Jones,517e070b-98ff-1a47-d062-66d2123a013d,Shopcaster,2011-10-01,NaN,True,Co-Founder & CTO,executive,1,2011-10-07,-0.2,2,0


In [145]:
founder_df_v2[founder_df_v2['experience_num_companies'] > 0]

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder,founded_on,experience_years,experience_num_companies,is_previously_successful
3,284395a7-916e-e815-5e88-c495ccc556c7,Sean Parker Founder @ Plaxo,1b0bb280-c870-8f14-6c95-7555cc5f3699,Sean Parker,91f5ca18-e026-9482-8e4f-22593943e0f5,Plaxo,2001-05-01,2004-04-01,False,Founder,executive,1,2001-01-01,1.4,1,0
6,6069e78a-a3fb-148b-7841-19b845f0f29d,Jason Johnson Co-Founder & Chief Executive Off...,3191bbf3-3e45-2c74-7087-f29d12466190,Jason Johnson,3d607a21-39bb-651e-c53f-9890ee334f18,Ygnition Networks,1997-12-01,2000-06-01,False,Co-Founder & Chief Executive Officer,executive,1,2001-01-01,1.0,1,0
7,535931ad-3f4d-5f91-3b08-d31f95f8bd99,Jonathan Abrams Founder @ Friendster,1d3b6baf-bbf5-e2dd-44c5-6a1b65ae2142,Jonathan Abrams,2a5fa1db-4075-04fb-d76c-372158a3a2c8,Friendster,2002-01-01,2005-01-01,False,Founder,executive,1,2001-01-03,2.0,2,0
8,f52a5e4d-b8fb-190f-548e-423161f6df96,Matt Bell Founder and Chief Scientist @ Reactrix,5b395a19-38a1-63e6-cb09-b8fc7ab2b694,Matt Bell,0f19649c-4fb9-cc37-d9b9-753d738c9da7,Reactrix,2002-01-01,2009-01-01,False,Founder and Chief Scientist,executive,1,2002-01-01,1.0,1,0
18,beeba46f-fbe8-03ba-2b8e-3f1ba9cbe223,"Bart Decrem Founder and CEO @ Flock, Inc.",d1b5edf4-8c87-a22c-def7-b7b1febfed4e,Bart Decrem,886dac3f-800a-9b51-c221-9df4133586fa,"Flock, Inc.",2005-01-01,2006-07-01,False,Founder and CEO,executive,1,2005-01-01,1.3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48486,69a2de0a-8f47-0ed2-2fca-aa7899926b7c,"Mathieu Le Roux Co-Founder, Brazilian Franchis...",d34a0656-3fbc-4256-edae-ea23f97a2745,Mathieu Le Roux,a1b7c1c3-c56f-9a72-7215-4e802c33bc67,Le Wagon,2016-04-01,NaN,True,"Co-Founder, Brazilian Franchise",executive,1,2013-08-28,0.0,1,0
48487,339da87c-b7e5-cb38-879b-47a8f3f280dc,Sébastien Saunier CTO & Co-Founder @ Le Wagon,d213c1c5-2876-01b1-e995-0959b28442cf,Sébastien Saunier,a1b7c1c3-c56f-9a72-7215-4e802c33bc67,Le Wagon,2014-01-01,NaN,True,CTO & Co-Founder,executive,1,2013-08-28,3.5,2,0
48511,994dfa34-6e29-44ae-9cff-bf7de925b5ca,Aleksander Popel Co-Founder & CSO @ AsclepiX T...,1c4bca13-bc42-4e5c-9f36-d79a9b691521,Aleksander Popel,6543be1e-bc0f-4837-aca9-c09a71b08f82,AsclepiX Therapeutics,2011-05-01,NaN,True,Co-Founder & CSO,executive,1,2011-01-01,0.0,1,0
48539,96b4eeed-7992-6a28-520d-ae8dab567cdc,Bob Bickel Founder @ RunSignUp,727dd828-87fc-5d9a-64c0-5467ced93a78,Bob Bickel,247e49e8-7b74-c3e9-a336-99751c791dda,RunSignUp,2009-08-01,NaN,True,Founder,executive,1,2010-01-01,33.7,9,0


In [92]:
founder_df_v2[founder_df_v2['experience_num_companies'] == 30]

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder,founded_on,experience_years,experience_num_companies,is_previously_successful
4336,5b84ace4-a9a0-4e54-8756-e18abc4c9722,Noubar Afeyan Co-Founder & Board Member @ Rubi...,396dc987-fa6b-d5db-c6da-487576633993,Noubar Afeyan,85aaa9d4-c8ed-fe27-3aac-a2b2eddaf80a,Rubius Therapeutics,2014-01-01,NaN,True,Co-Founder & Board Member,executive,1,2013-01-01,159.5,30,1


In [100]:
jobs_test = jobs_df_v3[jobs_df_v3['people_uuid'] == 'f77f7160-08f9-59d2-2ef1-e7ff761abaad']
jobs_test

,job_uuid,people_uuid,org_uuid,started_on,ended_on,is_current
933406,8bef4d21-db14-653c-ca58-9a9158c9839c,f77f7160-08f9-59d2-2ef1-e7ff761abaad,82ef43f0-9609-d345-e723-b7e12c421fc3,2011-01-01,NaT,True


In [101]:
jobs_test['ended_on'].max() - jobs_test['started_on'].min()

NaT

# Save final file

In [ ]:
len(main_processing_df_v4)

In [ ]:
main_processing_df_v4.head(3)

# Rename Column and Save File

In [ ]:
main_processing_df_v4.set_index('org_uuid',inplace=True)

In [ ]:
len(main_processing_df_v4)

In [ ]:
new_column_name = list()
for column in list(main_processing_df_v4.columns):
    new_column_name.append('people_' + column)
new_column_dict = dict(zip(main_processing_df_v4.columns,new_column_name))
main_processing_df_v4.rename(columns = new_column_dict,inplace=True)
main_processing_df_v4.head(3)

In [ ]:
main_processing_df_v4.to_csv(r'd:\msc-project\data\final\features_people.csv',encoding='utf=8')